In [2]:
pip install torch torchvision torchaudio

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     --------------------------------- ------ 51.2/60.8 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 60.8/60.8 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/203.1 MB 3.3 MB/s eta 0:01:02
   ---------------------------------------- 0.2/203.1 MB 3.1 MB/s eta 0:01:06
   ---------------------------------------- 0.4/203.1 MB 3.2 MB/s eta 0:01:04
   ---------------------------------------- 0.6/203.1 MB 3.2 MB/s eta 0:01:05
   ---------------------------------------- 0.7/203.1 MB 2.9 MB/s eta 0:01:10
   ---------------------------------------- 0.9/203.1 MB 3.1 MB/s eta 0:01:07
   ---------------------------------------- 1.0/203.1 MB 3.1 MB/s eta 0:01:06
   ---------------------------------------- 1.2/203.1 MB 3.1 MB/s eta 0:01:07
   ---------------------------------------- 1.3/203.1 MB 3.1 MB/s eta 0:0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install ultralytics


  Using cached ultralytics-8.3.63-py3-none-any.whl.metadata (35 kB)
  Using cached matplotlib-3.10.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached scipy-1.15.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached contourpy-1.3.1-cp311-cp311-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.55.3-cp311-cp311-win_amd64.whl.metadata (168 kB)
  Using cached kiwisolver-1.4.8-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
  Usin

In [5]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://download.pytorch.org/whl/cu113
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import torch
print(torch.__version__)  # Verify the version
print(torch.cuda.is_available())  # Should return True if CUDA is properly set up
print(torch.cuda.get_device_name(0))  # Should display the name of your GPU


2.5.1+cpu
False


AssertionError: Torch not compiled with CUDA enabled

In [1]:
pip install tensorflow

  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl.metadata (59 kB)
   ---------------------------------------- 0.0/390.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/390.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/390.0 MB 2.4 MB/s eta 0:02:43
   ---------------------------------------- 1.6/390.0 MB 3.2 MB/s eta 0:02:01
   ---------------------------------------- 2.4/390.0 MB 3.2 MB/s eta 0:02:02
   ---------------------------------------- 3.4/390.0 MB 3.7 MB/s eta 0:01:46
   ---------------------------------------- 4.5/390.0 MB 3.8 MB/s eta 0:01:42
    --------------------------------------- 5.0/390.0 MB 3.7 MB/s eta 0:01:44
    --------------------------------------- 5.8/390.0 MB 3.6 MB/s eta 0:01:46
    --------------------------------------- 6.3/390.0 MB 3.5 MB/s eta 0:01:49
    --------------------------------------- 7.1/390.0 MB 3.5 MB/s eta 0:01:49
    --------------------------------------- 7.6/390.0 MB 3.5 MB/s eta 0:01:50
    ------

In [2]:
import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
    print("GPU is available!")
    for gpu in tf.config.list_physical_devices('GPU'):
        print(f"GPU Name: {gpu.name}")
else:
    print("No GPU is available.")

No GPU is available.


In [2]:
pip install nvidia-cudnn

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [17 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\rohan\AppData\Local\Temp\pip-install-gtcgukw2\nvidia-cudnn_cca82fb312a248f5a24907f13451cd06\setup.py", line 150, in <module>
          raise RuntimeError(open("ERROR.txt", "r").read())
      RuntimeError:
      ###########################################################################################
      The package you are trying to install is only a placeholder project on PyPI.org repository.
      This package is hosted on NVIDIA Python Package Index.
      
      This package can be installed as:
      ```
      $ pip install nvidia-pyindex
      $ pip install nvidia-cudnn
      ```
      #######################################################################################

In [3]:
pip install nvidia-pyindex

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=8433 sha256=bcc142839b256ed552208f30095341bd257014367dae817c51014fab94d27a83
  Stored in directory: c:\users\rohan\appdata\local\pip\cache\wheels\39\63\71\c50214b560fa8c319598c2de3c1616f6d68e1d2c7f17a5e82d
Successfully built nvidia-pyindex
Note: you may need to restart the kernel to use updated packages.
